## Replica model

### Spatial reranking

Testing how to implement the spatial reranking.

In [1]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import sys

sys.path.insert(0, "../model/")
from utils import *
from metrics import *
from model_replica import ReplicaNet
from dataloader_replica import ReplicaDataset


path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'
replica_dir='/mnt/project_replica/datasets/cini/'



2022-04-16 17:34:06.531402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
